In [2]:
# Importamos las librerias que vamos a usar 
import pandas as pd
import json
import gzip
import ast


In [3]:
# Cremos una variable con la ruta del archivo
archivo = 'C:/Users/HP/Downloads/users_items.json.gz'
# Iniciamos una variable donde vamos a guarda la informacaion
data = []

# Abrimos el archivo y hacemos un for para cargar cada fila de los datos dentro de variable que definimos antes
with gzip.open(archivo, 'rt', encoding='utf-8') as file:
    for line in file:
        try:
            json_data= ast.literal_eval(line)
            data.append(json_data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Crear un DataFrame a partir de los datos JSON
df = pd.DataFrame(data)


In [4]:
# Borramos las variables que no usaremos 
df.drop(['user_url', 'steam_id'], axis= 1, inplace=True)

In [5]:
# Usamos el metodo .explote, ya que la columna de items es una coleccion, lo quenos hace el metodo explode es darnos
# Una fila del usuario por cada item en su coleccion
df_items = df.explode('items').reset_index(drop=True)
# Con pd.json_normalize convertimos cada lleve de las colecciones en columnas del para el dataframe 
items = pd.json_normalize(df_items['items'])

In [6]:
items

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6.0,0.0
1,20,Team Fortress Classic,0.0,0.0
2,30,Day of Defeat,7.0,0.0
3,40,Deathmatch Classic,0.0,0.0
4,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...
5170010,373330,All Is Dust,0.0,0.0
5170011,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,521570,You Have 10 Seconds 2,4.0,4.0
5170013,519140,Minds Eyes,3.0,3.0


In [7]:
# Aqui Agregamos el ide de usurio y la cantidas de items por usuario
items['users_id']= df_items['user_id']
items['items_count']= df_items['items_count']

In [8]:
# Borramos las columnas que no vamos a usar
items.drop('playtime_2weeks',axis=1, inplace= True)

In [9]:
# Borramos los NA e imprimimos los el tamaño del dataframe
items.dropna(inplace=True)
print(items.shape)

(5153209, 5)


In [10]:
# Seleccionamos las primeras primeras 25mil filas para hacer el dataframe mas manejable
items= items.iloc[:25000,:]
print(items.shape)

(25000, 5)


In [11]:
# Exportamos el dataframe en formato csv
items.to_csv('C:/Users/HP/Documents/Proyectos/PI_1_Mlops/Data/items.csv',index= False)